<a href="https://colab.research.google.com/github/lukasdekanovsky/Become_PythonDeveloper_AI/blob/main/REKURENT_(1D_CNN)_na%C5%A1ept%C3%A1va%C4%8D_dal%C5%A1%C3%ADch_znak%C5%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Textový generátor**

https://www.tensorflow.org/text/tutorials/text_generation

# 1) **Import knihoven**

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

# 2) **Stáhnutí datasetu - Shakespeare**

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


Přečtení dat - ukázka, základní charakteristika dat

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


Přečtení dat - ukázka do 250 charakteru

In [ ]:
print(text[:400])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it 


Přečtení dat - kolik máme charakterů

In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


# 3) **Textový preprocessing** - jeho vektorizace/ string na numerickou reprezentaci/ = tokenizace

**pomocí .StringLookup()**

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Nyní převedení všech 65 charakterů na tokeny

In [ ]:
# ids_from_chars  = token z chars (text)
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

A projekce na example_texts

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Nakonec bude provedena zpětná inverze tokenu na čtený text

In [ ]:
# chars_from_ids = navrací tokenu jeho textovou složku
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

pomocí **.reduce_join** můžeme spojit charaktery zpětně do stringu

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

**Shrnutí postupu:**
* 1) text = zadaný text v podobě stringu
* 2) tf.strings.unicode_split(text, input_encoding="UTF-8") - tokenizace
* 3) tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None) převde charakter na numerické ID
* 4) tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) převede zpět
* celková cesta proměnné: text -> chars -> ids_from_chars -> ids -> chars_from_ids -> chars

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# 4) **Prediční algoritmus?**

Pokud dáme algoritmu charaktr nebo jeho sekvenci, jaký bude pravděpodobně další charakter?
* Inputem modelu bude sekvence charakterů  a trénujeme output - následné charaktery


Rozdělíme si text do example sequences
* každý input bude obsahovat seq_lenght
* příklad seq_lenght = 4 a náš text je Hell - target sequence bude ello.
* použijeme **tf.data.Dataset.from_tensor_slices()** funkci ke konverzi textu do toku znakových indexů

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

Vezmeme prvních 10 znaků

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


Zadefinujeme seq_length na 100

In [ ]:
seq_length = 100

metoda **.batch**

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


**Sumarizace procesu spojením tokenů se stringem**

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Pro trénování budeme potřebovat datasety a) Input b) label v páru
* Input i label jsou sekvence

Zadefinujeme si proto funkci **split_input_target**
* Ta nám roztrhne seznam na input [:-1] = vše bez posledního písmena
* a target [1:] = vše bez prvního písmene





In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# 5) **Tvorba tréninkových batchů**

In [ ]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

# 6) **Tvorba modelu**

* **tf.keras.layers.Embedding** - input vrstva, bude mapovat každé charakter ID do vektoru o embedding_dim = dimenzích
* **tf.keras.layers.GRU** - typ RNN o velikosti rnn_units
* **tf.keras.layers.Dense** - output vrstva o vocab_size

In [ ]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Zkouška modelu

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


Jak predikuje bez tréninku

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([23, 17, 37, 14, 37, 54, 26,  8, 13,  5, 42, 20,  9,  5,  6, 24, 13,
       34, 62, 13, 53, 51, 51, 44, 64, 61, 19,  6, 25, 54, 22, 43, 59, 21,
        1, 43, 37, 22, 37, 51, 20, 26, 30, 65, 17, 39, 32, 20, 11, 32,  3,
       65, 29, 41, 54, 21, 52, 33, 11, 16, 34, 21, 65,  6,  1, 49, 30, 50,
       46, 65, 63, 17, 27,  0, 47, 52, 58, 48, 21, 30, 36, 56, 16, 15, 40,
       29, 36,  0,  3,  7, 57, 27, 15, 14, 59, 31, 24, 41, 63, 20])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b": you shall hear him.\n\nCORIOLANUS:\nHail, lords! I am return'd your soldier,\nNo more infected with my"

Next Char Predictions:
 b"JDXAXoM-?&cG.&'K?Uw?nlleyvF'LoIdtH\ndXIXlGMQzDZSG:S!zPboHmT:CUHz'\njQkgzxDN[UNK]hmsiHQWqCBaPW[UNK]!,rNBAtRKbxG"


# 7) **Trénink modelu**

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.190151, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

66.032776

Kompilace

In [ ]:
model.compile(optimizer='adam', loss=loss)

Tvorba checkpointů v průběhu tréninku

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Provedení tréninku

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 12s 52ms/step - loss: 2.7190
Epoch 2/20
172/172 [==============================] - 10s 52ms/step - loss: 1.9868
Epoch 3/20
172/172 [==============================] - 10s 53ms/step - loss: 1.7143
Epoch 4/20
172/172 [==============================] - 10s 54ms/step - loss: 1.5538
Epoch 5/20
172/172 [==============================] - 10s 54ms/step - loss: 1.4554
Epoch 6/20
172/172 [==============================] - 10s 53ms/step - loss: 1.3865
Epoch 7/20
172/172 [==============================] - 10s 54ms/step - loss: 1.3331
Epoch 8/20
172/172 [==============================] - 10s 54ms/step - loss: 1.2894
Epoch 9/20
172/172 [==============================] - 11s 55ms/step - loss: 1.2484
Epoch 10/20
172/172 [==============================] - 11s 56ms/step - loss: 1.2083
Epoch 11/20
172/172 [==============================] - 11s 56ms/step - loss: 1.1692
Epoch 12/20
172/172 [==============================] - 11s 56ms/step - loss: 1.1284
E

# 8) **Generování textu**

Nejjednoduší možností je spustit ho ve smyčce

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:

Provost:
I tale thy kiss, here burns to fear this face,
And bid Bianca may be prosperous,
and make spocl'd it. Another's runnows! So may for these I call?

KATHARINA:
If it begins to made me evils, in quickly house
our nails, but practised your sword, what I do be
Much here, I have for long wither'd.
O Lord, with futy join'd by this high discourse;
And say thou will, 'tis in my queen's are too ears:
Who, had he done, but he is Perutis.
By mine honesty, what were my suit ere he requires?
Then have my wife's body. But, for being sport in love, I am
your fully dishonour'd out of mine own,
Let what fearful service of his sight, as harshal, or
sweet bloody day's partelf,
And therefore, in meeting tear it conful.

BIANCA:
Faith, wife, sir, his sword, and harbship is Tybalt dead;
Or fiery con equal to hear of them.

KING EDWARD IV:
Is Clarence, Lord Aumertable, Lord prierse our father's son,
She was convey'd to dive in a hoop;
So much to show us in his shore;
With two summer's hand th

Improvement tips:
* The easiest thing you can do to improve the results is to train it for longer (try EPOCHS = 30).

* You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

* If you want the model to generate text faster the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

# 9) **Export generátoru**

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Tut, what my weapons, no?
Alas, thy counsels purgued up, I gave than sweet and
spit atiden, and rai


# 10) **Implementace tréninkové smyčky**

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 13s 51ms/step - loss: 2.7310
